## Import packages

In [1]:
from copy import deepcopy
import os

import numpy as np
import pandas as pd
import tifffile
from ipywidgets import IntSlider, VBox
import fastplotlib as fpl

from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

import mesmerize_core as mc
from mesmerize_core.arrays import LazyTiff
from mesmerize_viz import *

from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)
    
pd.options.display.max_colwidth = 120

## Load dataframe

In [2]:
mc.set_parent_raw_data_path(r"D:\Data\SingleCellData")
batch_path = mc.get_parent_raw_data_path().joinpath("mesmerize-cnmfe/batch.pickle")
df = mc.load_batch(batch_path)
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,s2371_240328,s2371_240328.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': e57b69d2-49fb-4f1d-b3cc-3b632555b6d9\e57b69d2-49fb-4f1d-b3cc-3b632555b6d9_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T17:32:25,785.84 sec,None,e57b69d2-49fb-4f1d-b3cc-3b632555b6d9
1,mcorr,s2371_240404,s2371_240404.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': f9267f04-82fc-4635-a0be-812e1cc4d7ed\f9267f04-82fc-4635-a0be-812e1cc4d7ed_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T18:02:39,1813.2 sec,None,f9267f04-82fc-4635-a0be-812e1cc4d7ed
2,mcorr,s2371_240405,s2371_240405.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 3a8d6f2b-0410-4e54-86d1-7264ad08d95e\3a8d6f2b-0410-4e54-86d1-7264ad08d95e_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T18:33:47,1867.88 sec,None,3a8d6f2b-0410-4e54-86d1-7264ad08d95e
3,mcorr,s2371_240411,s2371_240411.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 8cbb85ac-0d42-4fda-a03b-ca6f1e36d58d\8cbb85ac-0d42-4fda-a03b-ca6f1e36d58d_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T19:01:17,1648.76 sec,None,8cbb85ac-0d42-4fda-a03b-ca6f1e36d58d
4,mcorr,s2371_240412,s2371_240412.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 95679105-aecd-42ad-ac24-aa6f3d45ae8f\95679105-aecd-42ad-ac24-aa6f3d45ae8f_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T19:15:07,829.27 sec,None,95679105-aecd-42ad-ac24-aa6f3d45ae8f
5,mcorr,s2371_240417,s2371_240417.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': eb507a79-243a-4870-a1c1-874d9fab14b7\eb507a79-243a-4870-a1c1-874d9fab14b7_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T19:38:44,1416.66 sec,None,eb507a79-243a-4870-a1c1-874d9fab14b7
6,mcorr,s2372_240328,s2372_240328.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 7c42162f-db8c-4aa0-831f-c22184fc050e\7c42162f-db8c-4aa0-831f-c22184fc050e_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T19:57:13,1108.8 sec,None,7c42162f-db8c-4aa0-831f-c22184fc050e
7,mcorr,s2372_240404,s2372_240404.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 76d885fe-0777-4b28-b84d-1db128a09683\76d885fe-0777-4b28-b84d-1db128a09683_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T20:16:56,1182.66 sec,None,76d885fe-0777-4b28-b84d-1db128a09683
8,mcorr,s2372_240405,s2372_240405.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 4b3d939b-6e53-43bc-b496-1b1bfc8b878c\4b3d939b-6e53-43bc-b496-1b1bfc8b878c_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T20:34:04,1027.57 sec,None,4b3d939b-6e53-43bc-b496-1b1bfc8b878c
9,mcorr,s2372_240411,s2372_240411.tif,"{'main': {'gSig_filt': (5, 5), 'pw_rigid': True, 'max_shifts': (5, 5), 'strides': (48, 48), 'overlaps': (24, 24), 'm...",{'mean-projection-path': 04f448c0-352b-4ce0-b2fc-719025fbe1ba\04f448c0-352b-4ce0-b2fc-719025fbe1ba_mean_projection.n...,2024-04-19T17:19:07,2024-04-19T20:50:28,982.85 sec,None,04f448c0-352b-4ce0-b2fc-719025fbe1ba


## Add new tif file

In [ ]:
movie_path = mc.get_parent_raw_data_path().joinpath("s2376_240405_240411.tif")

## Motion correction

In [ ]:
params =\
{
    "main":
    {
        "gSig_filt": (7, 7), # a gSig_filt value that brings out "landmarks" in the movie
        "pw_rigid": False,
        "max_shifts": (25, 25),
        "strides": (48, 48),
        "overlaps": (24, 24),
        "max_deviation_rigid": 3,
        "border_nan": "copy",
    }
}

In [ ]:
df.caiman.add_item(
    algo="mcorr",
    input_movie_path=movie_path,
    params=params,
    item_name=movie_path.stem
)

df

In [ ]:
df.iloc[57].caiman.run()
# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

In [3]:
viz = df.mcorr.viz()
viz.show()

C:\Users\prsalab\miniforge3\envs\mescore\lib\site-packages\ipydatagrid\datagrid.py:460: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

C:\Users\prsalab\miniforge3\envs\mescore\lib\site-packages\fastplotlib\graphics\_features\_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


## CNMFE

In [ ]:
gSig = 2
corr_pnr = {'min_corr':0.85, 'min_pnr':12}

In [ ]:
params_cnmfe =\
{
    "main":
    {
        'method_init': 'corr_pnr',  # use this for 1 photon
        'K': None,
        'gSig': (gSig, gSig),
        'gSiz': (4 * gSig + 1, 4 * gSig + 1),
        'merge_thr': 0.7,
        'p': 1,
        'tsub': 2,
        'ssub': 1,
        'rf': 40,
        'stride': 20,
        'only_init': True,    # set it to True to run CNMF-E
        'nb': 0,
        'nb_patch': 0,
        'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
        'low_rank_background': None,
        'update_background_components': True,  # sometimes setting to False improve the results
        'normalize_init': False,               # just leave as is
        'center_psf': True,                    # leave as is for 1 photon
        'ssub_B': 2,
        'ring_size_factor': 1.4,
        'del_duplicates': True,                # whether to remove duplicates from initialization
        **corr_pnr # unpack corr_pnr vals into here
    }
}

In [ ]:
df.caiman.add_item(
    algo="cnmfe",
    input_movie_path=df.iloc[0],
    params=params_cnmfe,
    item_name=df.iloc[0]["item_name"]
)

df

In [ ]:
df.iloc[-1].caiman.run()
# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

In [3]:
viz_cnmf = df.cnmf.viz(
    image_data_options=["input", "rcm"], # cnmfe does not support rcb and residuals yet
)
viz_cnmf.show(sidecar = True)

C:\Users\prsalab\miniforge3\envs\mescore\lib\site-packages\ipydatagrid\datagrid.py:460: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

RFBOutputContext()

C:\Users\prsalab\miniforge3\envs\mescore\lib\site-packages\mesmerize_viz\_cnmf.py:621: RuntimeWarning: overflow encountered in scalar multiply
  size = rcm.shape[0] * rcm.shape[1] * rcm.shape[2]
C:\Users\prsalab\miniforge3\envs\mescore\lib\site-packages\fastplotlib\graphics\_features\_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


RFBOutputContext()

In [ ]:
viz_cnmf.image_widget.cmap = 'gray'

In [ ]:
dir(viz_cnmf.cnmf_obj)

In [ ]:
from fastplotlib.widgets import ImageWidget
rcm = df.iloc[-1].cnmf.get_rcm(component_indices="good")

# view with ImageWidget
iw = ImageWidget(data=rcm)
iw.show()

In [ ]:
df.iloc[-1].cnmf.get_good_components()

## Check gSig_filt

In [4]:
movie_path = mc.get_parent_raw_data_path().joinpath("s2376_240328.tif")
input_movie = tifffile.imread(movie_path)
slider_gsig_filt = IntSlider(value=3, min=1, max=33, step=2,  description="gSig_filt")

def apply_filter(frame):
    # read slider value
    gSig_filt = (slider_gsig_filt.value, slider_gsig_filt.value)
    
    # apply filter
    return high_pass_filter_space(frame, gSig_filt)

# we can use frame_apply feature of `ImageWidget` to apply 
# the filter before displaying frames
funcs = {
    # data_index: function
    1: apply_filter  # filter shown on right plot, index 1
}

# input movie will be shown on left, filtered on right
iw_gs = fpl.ImageWidget(
    data=[input_movie, input_movie.copy()],
    frame_apply=funcs,
    names=["raw", "filtered"],
    grid_plot_kwargs={"size": (1200, 600)},
    cmap="gnuplot2"
)

def force_update(*args):
    # kinda hacky but forces the images to update 
    # when the gSig_filt slider is moved
    iw_gs.current_index = iw_gs.current_index
    iw_gs.reset_vmin_vmax()

iw_gs.reset_vmin_vmax()
    
slider_gsig_filt.observe(force_update, "value")

VBox([iw_gs.show(), slider_gsig_filt])

RFBOutputContext()

## Check corr and pnr

In [ ]:
# get the motion corrected output, this is a memmap array
mcorr_movie = df.iloc[7].mcorr.get_output()

gSig = 3
corr, pnr = correlation_pnr(mcorr_movie[::2], gSig=gSig, swap_dim=False)
# to show the correlation and pnr images
iw_corr_pnr = fpl.ImageWidget(
    [corr, pnr], 
    names=["corr", "pnr"],
    grid_plot_kwargs={"size": (650, 300)},
    cmap="turbo",
)

# mcorr vids, we will display thresholded mcorr vids
mcorr_vids = [mcorr_movie.astype(np.float32) for i in range(4)]

# sync the threshold image widget with the corr-pnr plot


iw_thres_movie = fpl.ImageWidget(
    mcorr_vids, 
    names=["over corr threshold", "over pnr threshold", "under corr threshold", "under pnr threshold"],
    # sync this with the corr-pnr plot
    cmap="gnuplot2"
)

# display threshold of the spatially filtered movie
def spatial_filter(frame):
    f = high_pass_filter_space(frame, (3, 3))
    return f


# threshold
def threshold(frame, mask):
    # optionally use spatial filter
    t = spatial_filter(frame)
    
    t = t.copy()
    
    t[mask] = t.min()
    
    return t

# Set the thresholded images using the vmin set from top subplots
# dict of threshold lambda wrappers to set on ImageWidget
# this sets the frame_apply for each subplot
threshold_funcs = {
    0: lambda frame: threshold(frame, corr < iw_corr_pnr.gridplot["corr"].graphics[0].cmap.vmin),
    1: lambda frame: threshold(frame, pnr < iw_corr_pnr.gridplot["pnr"].graphics[0].cmap.vmin),
    2: lambda frame: threshold(frame, corr > iw_corr_pnr.gridplot["corr"].graphics[0].cmap.vmin),
    3: lambda frame: threshold(frame, pnr > iw_corr_pnr.gridplot["pnr"].graphics[0].cmap.vmin)
}

# set the dict of lambda wrappers
iw_thres_movie.frame_apply = threshold_funcs

# update threshold plots when the corr pnr sliders move
def update_threshold_plots(*args):
    iw_thres_movie.current_index = iw_thres_movie.current_index

# this will get easier in the future
iw_corr_pnr.gridplot["corr"].docks["right"]["histogram_lut"].linear_region.selection.add_event_handler(update_threshold_plots)
iw_corr_pnr.gridplot["pnr"].docks["right"]["histogram_lut"].linear_region.selection.add_event_handler(update_threshold_plots)

VBox([iw_corr_pnr.show(), iw_thres_movie.show()])